<a href="https://colab.research.google.com/github/katemartin99/Lab-6-1/blob/main/1_4_health_care_survey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import polars as pl

# Why use relative addresses?

In this notebook, we will illustrate

1. That relative addresses for loading data files works, but
2. Using absolute addresses for loading data files *will not*.

## Problem 1 - Load the `lat_long_example.csv` file using a relative address.

**Tasks.**
1. Open a terminal, start `nu`, and navigate to the root menu of your first/primary data repository,
2. Use `ls **/*` to get the relative address of `lat_long_example.csv`, and
3. Use `polars` to load and inspect these data using this relative path.

In [2]:
import pyarrow

In [9]:
relative_path = "./data/lat_long_examples.csv"

In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [15]:
!mkdir -p sample_data

!curl https://raw.githubusercontent.com/katemartin99/Lab-6-1/refs/heads/main/data/lat_long_examples.csv \
     -o ./sample_data/lat_long_examples.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   206  100   206    0     0    559      0 --:--:-- --:--:-- --:--:--   558


In [16]:
(lat_long_example :=
 spark.read.csv('./sample_data/lat_long_examples.csv', header=True)
)

DataFrame[City 1: string, Lat 1: string, Long 1: string, City 2: string, Lat 2: string, Long 2: string, Distance from Web (km): string]

## Problem 2 - Load the `lat_long_example.csv` file using a absolute address.

**Tasks.**
1. Open a terminal, start `nu`, and navigate to the root menu of *one of your first/primary data repository,
2. Use `glob **/*` to get the absolute address of `lat_long_example.csv`, and
3. Use `polars` to load and inspect these data.

In [17]:
absolute_path = "/Users/tp5521zg/Desktop/Lab-6-1/data/lat_long_examples.csv"

## Illustrating the problem with absolute addresses

While the relative address in problem 1 points to the data IN THIS COPY of the repo, the absolute address points to the data in EXACTLY one of the copies of the repository. This becomes a problem if (A) anything changes in that repository, or (B) we are working on a different machine.

**Tasks.** To illustrate why this is a problem, do the following.

1. From your first/primary repository commit and push this notebook to GitHub,
2. Fetch and pull this notebook to another local copy of the repository,
3. In your file explorer (Files or Finder), move your first/primary repository into another folder, e.g., make a new folder and drag-and-drop the repo.
4. Rerun the cells in each local copy of the repository and document your findings in the WORD document.

In [18]:
!curl https://raw.githubusercontent.com/katemartin99/Lab-6-1/refs/heads/main/data/health_survey_v2.csv \
     -o ./sample_data/health_survey_v2.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  195k  100  195k    0     0   518k      0 --:--:-- --:--:-- --:--:--  519k


In [19]:
(health_survey_v2 :=
 spark.read.csv('./sample_data/health_survey_v2.csv', header=True)
)

DataFrame[ID: string, F1: string, F5: string, F2: string, F1_1: string, F2_1: string, F6: string, F4: string, F3: string, F5_1: string, F1_2: string, F2_2: string, F6_1: string, F2_3: string, F4_1: string, F2_4: string, F5_2: string, F2_5: string, F6_2: string, F1_3: string, F2_6: string, F5_3: string, F4_2: string, F2_7: string, F3_1: string, F2_8: string, F5_4: string, F3_2: string, F1_4: string, F3_3: string, F1_5: string, F5_5: string, F6_3: string, F1_6: string, F5_6: string, F2_9: string, F3_4: string, F4_3: string, F2_10: string, F1_7: string, F6_4: string, F4_4: string, F5_7: string, F3_5: string, F2_11: string]

In [20]:
!curl https://raw.githubusercontent.com/katemartin99/Lab-6-1/refs/heads/main/data/ReverseCodingItems_v2.csv \
     -o ./sample_data/ReverseCodingItems_v2.csv.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3157  100  3157    0     0   8777      0 --:--:-- --:--:-- --:--:--  8793


In [22]:
!ls sample_data


anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md
health_survey_v2.csv	      ReverseCodingItems_v2.csv.csv
lat_long_examples.csv


In [23]:
(ReverseCodingItems_v2 :=
 spark.read.csv('./sample_data/ReverseCodingItems_v2.csv.csv', header=True)
)

DataFrame[Question: string, Construct: string, Question # on Qualtrics Survey: string, Needs Reverse Coding?: string, Column Name: string]

In [24]:
from pyspark.sql.functions import col, explode, array, lit
survey_no_id = health_survey_v2.drop('ID')

melted = survey_no_id.selectExpr("stack({0}, {1}) as (variable, value)".format(
    len(survey_no_id.columns),
    ", ".join([f"'{c}', `{c}`" for c in survey_no_id.columns])
))
unique_values = melted.select("value").distinct()
value_list = [row["value"] for row in unique_values.collect()]

In [25]:
reg = {
    'Strongly Agree': 5,
    'Somewhat Agree': 4,
    'Strongly Disagree': 3,
    'Neither Agree nor Disagree': 2,
    'Somewhat Disagree': 1
}

In [26]:
rev = {'Strongly Agree':1,
         'Somewhat Agree':2,
         'Strongly Disagree':3,
         'Neither Agree nor Disagree':4,
         'Somewhat Disagree':5,
        }

In [27]:
q_col = list(filter(lambda c: c != 'ID', health_survey_v2.columns))
print(q_col)

['F1', 'F5', 'F2', 'F1_1', 'F2_1', 'F6', 'F4', 'F3', 'F5_1', 'F1_2', 'F2_2', 'F6_1', 'F2_3', 'F4_1', 'F2_4', 'F5_2', 'F2_5', 'F6_2', 'F1_3', 'F2_6', 'F5_3', 'F4_2', 'F2_7', 'F3_1', 'F2_8', 'F5_4', 'F3_2', 'F1_4', 'F3_3', 'F1_5', 'F5_5', 'F6_3', 'F1_6', 'F5_6', 'F2_9', 'F3_4', 'F4_3', 'F2_10', 'F1_7', 'F6_4', 'F4_4', 'F5_7', 'F3_5', 'F2_11']


In [28]:
health_survey_v2.columns

['ID',
 'F1',
 'F5',
 'F2',
 'F1_1',
 'F2_1',
 'F6',
 'F4',
 'F3',
 'F5_1',
 'F1_2',
 'F2_2',
 'F6_1',
 'F2_3',
 'F4_1',
 'F2_4',
 'F5_2',
 'F2_5',
 'F6_2',
 'F1_3',
 'F2_6',
 'F5_3',
 'F4_2',
 'F2_7',
 'F3_1',
 'F2_8',
 'F5_4',
 'F3_2',
 'F1_4',
 'F3_3',
 'F1_5',
 'F5_5',
 'F6_3',
 'F1_6',
 'F5_6',
 'F2_9',
 'F3_4',
 'F4_3',
 'F2_10',
 'F1_7',
 'F6_4',
 'F4_4',
 'F5_7',
 'F3_5',
 'F2_11']

In [30]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


In [33]:
import pyspark.sql.functions as F

long_survey = (
    health_survey_v2
        .unpivot(
            ids='ID',
            values=q_col,
            variableColumnName='Question',
            valueColumnName='Response'
        )
        .alias("s")
)

reg_map = F.create_map([F.lit(x) for kv in reg.items() for x in kv])
rev_map = F.create_map([F.lit(x) for kv in rev.items() for x in kv])

coded = (
    long_survey
        .withColumn("reg_coding", reg_map[F.col("s.Response")])
        .withColumn("rev_coding", rev_map[F.col("s.Response")])
)

joined = (
    coded.alias("s")
         .join(
             ReverseCodingItems_v2.alias("r"),
             F.col("s.Question") == F.col("r.`Column Name`"),
             how="left"
         )
)

with_coding = (
    joined.withColumn(
        "coding",
        F.when(F.col("r.`Needs Reverse Coding?`") == "Yes",
               F.col("s.rev_coding"))
         .otherwise(F.col("s.reg_coding"))
    )
)

typed = (
    with_coding.withColumn(
        "q_type",
        F.regexp_replace(F.col("s.Question"), r"_.*", "")
    )
)

final_df = (
    typed.groupBy("ID")
         .pivot("q_type")
         .sum("coding")
         .limit(5)
         .toPandas()
)

final_df



,ID,F1,F2,F3,F4,F5,F6
0,125,32,46,22,20,31,20
1,124,37,55,21,18,32,21
2,7,35,56,20,21,28,22
3,51,33,52,24,17,24,23
4,169,35,49,17,22,28,20


In [36]:
import pyspark.sql.functions as F

long_df = (
    health_survey_v2
        .unpivot(
            ids="ID",
            values=q_col,
            variableColumnName="Question",
            valueColumnName="Response"
        )
        .alias("s")
)

reg_map = F.create_map([F.lit(x) for kv in reg.items() for x in kv])
rev_map = F.create_map([F.lit(x) for kv in rev.items() for x in kv])

coded_df = (
    long_df
        .withColumn("reg_coding", reg_map[F.col("s.Response")])
        .withColumn("rev_coding", rev_map[F.col("s.Response")])
)

joined_df = (
    coded_df.alias("s")
        .join(
            ReverseCodingItems_v2.alias("r"),
            F.col("s.Question") == F.col("r.`Column Name`"),
            how="left"
        )
)

with_coding = (
    joined_df.withColumn(
        "coding",
        F.when(F.col("r.`Needs Reverse Coding?`") == "Yes",
               F.col("s.rev_coding"))
         .otherwise(F.col("s.reg_coding"))
    )
)

typed_df = (
    with_coding.withColumn(
        "q_type",
        F.regexp_replace(F.col("s.Question"), r"_.*", "")
    )
)

result = (
    typed_df
        .groupBy("ID")
        .pivot("q_type")
        .sum("coding")
)

result = result.limit(5).toPandas()
result


,ID,F1,F2,F3,F4,F5,F6
0,125,32,46,22,20,31,20
1,124,37,55,21,18,32,21
2,7,35,56,20,21,28,22
3,51,33,52,24,17,24,23
4,169,35,49,17,22,28,20


In [37]:
import pyspark.sql.functions as F

long_df = (
    health_survey_v2
        .unpivot(
            ids="ID",
            values=q_col,
            variableColumnName="Question",
            valueColumnName="Response"
        )
        .alias("s")
)

reg_map = F.create_map([F.lit(x) for kv in reg.items() for x in kv])
rev_map = F.create_map([F.lit(x) for kv in rev.items() for x in kv])

coded_df = (
    long_df
        .withColumn("reg_coding", reg_map[F.col("s.Response")])
        .withColumn("rev_coding", rev_map[F.col("s.Response")])
)

reverse_clean = (
    ReverseCodingItems_v2.drop("Question")
    if "Question" in ReverseCodingItems_v2.columns
    else ReverseCodingItems_v2
).alias("r")

joined_df = (
    coded_df.alias("s")
        .join(
            reverse_clean,
            F.col("s.Question") == F.col("r.`Column Name`"),
            how="left"
        )
)

with_coding = (
    joined_df.withColumn(
        "coding",
        F.when(F.col("r.`Needs Reverse Coding?`") == "Yes",
               F.col("s.rev_coding"))
         .otherwise(F.col("s.reg_coding"))
    )
)

typed_df = (
    with_coding.withColumn(
        "q_type",
        F.regexp_replace(F.col("s.Question"), r"_.*", "")
    )
)

health_survey_summary_df = (
    typed_df
        .groupBy("ID")
        .pivot("q_type")
        .sum("coding")
)

health_survey_summary_df.limit(10).toPandas()


,ID,F1,F2,F3,F4,F5,F6
0,125,32,46,22,20,31,20
1,124,37,55,21,18,32,21
2,7,35,56,20,21,28,22
3,51,33,52,24,17,24,23
4,169,35,49,17,22,28,20
5,205,39,43,11,19,30,20
6,54,38,58,24,21,36,25
7,15,26,46,12,21,25,23
8,234,27,50,9,16,30,18
9,232,40,59,23,24,32,21
